<a href="https://colab.research.google.com/github/mugalan/energy-plus-utility/blob/dev/EMS_Cookbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#HVAC Controller Simulation and Online Parameter Identification Using EKF/RL (Kethaka)


## Overview

This project aims to create an AI-assisted, simulation-driven framework for intelligent building HVAC control and characterization. First, we will build a multi-zone HVAC modeling and simulation platform on top of EnergyPlus. This platform will support natural-language interaction for configuring buildings, HVAC systems, and schedules, and provide a user-friendly interface for visualizing simulation outputs (temperature, humidity, $CO_2$, loads, flows, etc.).

Using that platform, we will develop an Extended Kalman Filter (EKF)–based online parameter estimation method to identify key unknown building/zone parameters in real time. The target parameters include: (i) effective sensible thermal capacitance, (ii) overall heat transfer conductance to ambient, (iii) zone occupancy, and (iv) infiltration / outdoor air mass flow rates. We will verify the EKF estimator entirely in simulation by injecting known “ground truth” parameters into EnergyPlus and checking recovery accuracy and convergence.

Beyond the EKF, we will explore alternative adaptive/learning strategies for online estimation of the same parameters (for example, recursive least squares variants or data-driven/ML estimators), and compare their performance and robustness.

Finally, we will experimentally validate the estimator on a controlled laboratory-scale single-zone HVAC test rig. The goal is to demonstrate that the identified parameters track real physical behavior and can serve as a foundation for future decentralized multizone HVAC control and optimization.


## Objectives

* Develop a multi-zone HVAC modeling and simulation platform that is based on **EnergyPlus**. A conversationally assited platform for interactively building the model and convinient user interface for simuation data visualization.

* Develop a EKF based parameter estimation method to extract unknown model parameters such as effective sensible thermal capacitance, overall heat transfer conductance, occupancy, and infiltrated air mass flowrates.

* Simulation verification of the EKF using the developed simulation platform.

* Explore other other learning strtegies for the online estimation of unknown paramters such as effective sensible thermal capacitance, overall heat transfer conductance, occupancy, and infiltrated air mass flowrates.

* Experimental validation of the parameter estimation on a laboratory scale single zone HVAC setup



## References (Selected)


**EKF‑based Estimation in Buildings**  
3. **Madsen, H., & co‑authors.** “Grey‑box modeling and Kalman filtering for building thermal dynamics and parameter estimation.” (Various works, e.g., DTU Technical University reports and journal articles, 2000s–2010s.)  
4. **Wang, S., & Chen, Q.** “$CO_2$‑based occupancy estimation and ventilation control: modeling and state estimation approaches.” *Building and Environment*, ~2012 (methods include state‑space estimation such as EKF variations).

**Simulation Engine**  
5. **EnergyPlus Documentation.** *Engineering Reference* and *Input Output Reference*. U.S. DOE/ORNL/NREL. Available at: <https://energyplus.net/documentation>




#Decentralized HVAC Controller Development and Experimental Validation (Janith)

## Overview

This 28-week undergraduate project develops and validates a **decentralized HVAC control system** for a multi-zone building. Each zone runs a local controller that regulates **temperature**, **humidity ratio**, and **$CO_2$** concentration using zone **mass flow rate** commands, while a lightweight **central Air‑Handling Unit (AHU) coordinator** chooses AHU setpoints (supply air temperature, humidity ratio, and $CO_2$) to meet aggregate load and ventilation requirements.

Simulation is performed in **EnergyPlus**, with final **bench‑scale experimental verification** on a single‑zone testbed.

---

## Objectives

1. **Control**: Design per‑zone controllers that drive zone states into a comfort/safety set while minimizing energy use.
2. **Coordination**: Implement a central AHU policy that chooses $(T_{sa}, \omega_{sa}, c_{sa})$ based on the zones' aggregate thermal/moisture/IAQ needs.
4. **Validation**: Demonstrate closed‑loop performance and energy savings in EnergyPlus; verify modeling/estimation on a single‑zone physical rig.

---

## Problem Setting & Notation

For each zone $z \in \{1,\dots,N\}$, define the zone state
\begin{align}
\mathbf{x}_z(t) \;\triangleq\; \begin{bmatrix} T_z(t) \\ \omega_z(t) \\ c_z(t) \end{bmatrix},
\qquad
\mathbf{x}_z(t) \in \mathscr{D},
\end{align}
where the admissible set $\mathscr{D}$ encodes comfort/IAQ limits:
\begin{align}
\mathscr{D} \;=\; \Big\{ (T_z,\omega_z,c_z)\; \Big|\; T_{\min}\le T_z \le T_{\max},\; \omega_{\min}\le \omega_z \le \omega_{\max},\; c_z \le c_{\max} \Big\}.
\end{align}

**Control inputs** are zone **mass flow rates** $\dot m_{sa,z}(t)$ supplied from an AHU delivering air at $(T_{sa}(t), \omega_{sa}(t), c_{sa}(t))$. The AHU variables are coordinated centrally.

A simplified, control‑oriented continuous‑time model for thermal/moisture/$CO_2$ dynamics is
\begin{align}
\dot T_z = \frac{1}{C_{T,z}}\Big( U_z(T_o-T_z) + \dot m_{sa,z}c_p(T_{sa}-T_z) + Q^{\mathrm{int}}_{T,z} \Big),
\end{align}
\begin{align}
\dot \omega_z = \frac{1}{C_{\omega,z}}\Big( k_z(\omega_o-\omega_z) + \dot m_{sa,z}(\omega_{sa}-\omega_z) + Q^{\mathrm{int}}_{\omega,z} \Big),
\end{align}
\begin{align}
\dot c_z = \frac{1}{V_z}\Big( \dot m_{sa,z}(c_{sa}-c_z) + \dot m^{\mathrm{inf}}_z(c_o-c_z) + q^{\mathrm{occ}}_z \Big).
\end{align}
Unknowns such as $C_{T,z}, C_{\omega,z}, U_z, k_z, q^{\mathrm{occ}}_z$ are **estimated online** (EKF).

**Goal**: choose $\dot m_{sa,z}(t)$ and AHU setpoints $(T_{sa}, \omega_{sa}, c_{sa})$ to minimize energy use (fan + coil/pump surrogates) **subject to** $\mathbf{x}_z(t)\in\mathscr{D}$ and equipment limits.

---

## System Architecture

- **Zone Controllers (decentralized):** Each zone computes $\dot m_{sa,z}$ using local measurements $(T_z,\omega_z,c_z)$, forecasts (optional), and EKF state/parameter estimates. Controllers can be PI/MPC with soft comfort constraints.
- **AHU Coordinator (centralized, lightweight):** Aggregates zone demands to pick $(T_{sa},\omega_{sa},c_{sa})$ and total supply flow $\sum_z \dot m_{sa,z}$. Examples: rule‑based “cooling‑dominant/heating‑dominant” logic or small convex program.
- **Estimator (per zone):** EKF/UKF estimates thermal capacity, effective envelope conductance, and occupancy‑related gains from data (T, RH, $CO_2$).
- **Supervisor:** Enforces safety limits, fault flags, and fallbacks (e.g., revert to baseline schedules if estimates diverge).

---

## Research Questions

1. **Decentralization vs. performance:** How close can zone‑wise controllers get to a centralized optimum with only minimal coordination?
2. **Robustness:** How sensitive is performance to modeling error, weather disturbances, and actuator limits?
3. **Observability:** What minimal sensing (T, RH, $CO_2$, flows) yields reliable EKF parameter/occupancy estimates?
4. **AHU policy:** What simple policies for $(T_{sa},\omega_{sa},c_{sa})$ work well with diverse zone needs?

---

## Metrics & Evaluation

- **Comfort/IAQ compliance**: fraction of time $\mathbf{x}_z(t)\in \mathscr{D}$.  
- **Energy surrogate**: fan power $ \propto \big(\sum_z \dot m_{sa,z}\big)^\alpha$, coil loads $\propto \dot m_{sa}(T_{mix}-T_{sa})$, latent $\propto \dot m_{sa}(\omega_{mix}-\omega_{sa})$.  
- **Stability/robustness**: boundedness under forecast/model error; constraint violations (count, magnitude).  
- **Estimator quality**: parameter RMSE, occupancy estimation error, innovation whiteness.


---

## Resources

- EnergyPlus with Python API (runtime callbacks for reading/writing actuators & variables).  
- Standard Python stack (NumPy/SciPy, plotting, optimization).  
- Single‑zone testbed (ducted fan, heating/cooling source, T/RH/$CO_2$ sensors, DAQ).

---

## Expected Outcomes

- A working **decentralized HVAC** controller with a simple **AHU coordinator**.  

---

## References (Selected)

**Decentralized / Distributed HVAC Control**  
1. **Ma, Y., Kelman, A., Daly, A., & Borrelli, F.** “Distributed Model Predictive Control for Building Temperature Regulation.” *American Control Conference (ACC)*, 2012.  
2. **Dounis, A. I., & Caraiscos, C.** “Advanced control systems engineering for energy and comfort management in a building environment—A review.” *Renewable and Sustainable Energy Reviews*, 2012.
3. **Yang, Y., Srinivasan, S., Hu, G., & Spanos, C. J.** (2021). "Distributed Control of Multi-Zone HVAC Systems Considering Indoor Air Quality." arXiv:2003.08208.


**Simulation Engine**  
5. **EnergyPlus Documentation.** *Engineering Reference* and *Input Output Reference*. U.S. DOE/ORNL/NREL. Available at: <https://energyplus.net/documentation>

> The list above provides high‑impact entry points. During the project, refine with the exact editions/DOIs most aligned to your chosen model structures and estimation variants.


#HVAC Modelling/Estimation/Control

## Literature

* https://www.mdpi.com/1996-1073/16/20/7124
* https://www.mdpi.com/2071-1050/17/5/1955
* https://discovery.ucl.ac.uk/id/eprint/10116413/1/manuscript%20baycal.pdf
* https://arxiv.org/pdf/2508.09118#:~:text=Page%202,(7)
* https://link.springer.com/article/10.1007/s12273-025-1300-4
* https://www.mdpi.com/2075-5309/13/2/314?
* (Bayes) https://www.sciencedirect.com/science/article/pii/S0306261925013017
* (Distributed) https://arxiv.org/pdf/2003.08208

## Model

A compact state-space model for a single zone with temperature, moisture, and $CO_2$ dynamics.


### **Zone Temperature (Sensible Energy)**
\begin{align}
C_s \dot T_z =
- UA\,T_z - c_{pa}(m_{inf}+m_{sa})T_z
\ +\ UA\,T_o + c_{pa} m_{inf} T_o + c_{pa} m_{sa} T_{sa}
+ Q_{bg} + f_c\,q^{occ}_{sens} N
\end{align}
where:
- $C_s$ = effective sensible thermal capacitance $[J/K]$
- $UA$ = overall heat transfer conductance $[W/K]$
- $c_{pa}$ = specific heat of air $[\approx 1006~J/(kg\,K)]$
- $m_{inf}$, $m_{sa}$ = outdoor- infiltration/supply air flow rates $[kg/s]$
- $T_o$, $T_z$, $T_{sa}$ = outdoor, zone, supply air temperatures $[^\circ C]$
- $Q_{bg}$, $q^{occ}_{sens}$ = background and per-person sensible heat gains $[W]$
- $f_c$ = convective fraction of sensible internal gain
- $N$ = number of occupants


Define
\begin{align}
\alpha_{o}&=\frac{UA + c_{pa} m_{inf}}{C_s}\\
\alpha_s&=\frac{c_{pa}}{C_s}\\
\alpha_e&= \frac{Q_{bg} + f_c\,q^{occ}_{sens} N}{C_s}
\end{align}

Then we have
\begin{align}
\dot T_z =
- (\alpha_o+m_{sa}\alpha_s)T_z+\ \alpha_o T_o + m_{sa}\alpha_s T_{sa}
+ \alpha_e
\end{align}


***

### **Zone Humidity Ratio (Moisture)**
\begin{align}
M\dot\omega_z =
- (m_{inf} + m_{sa}) \omega_z
\ +\ m_{inf} \omega_o + m_{sa} \omega_{sa}
 + G_{bg} + g^{occ}_{\omega} N
\end{align}
where:
- $M$ = zone dry air mass or effective moisture capacity $$[kg_{dry}]$$
- $\omega_o,\, \omega_z,\, \omega_{sa}$ = outdoor, zone, supply air humidity ratios $[kg/kg_{dry}]$
- $G_{bg}$, $g^{occ}_{\omega}$ = background and per-person vapor gains $$[kg/s]$$


Define
\begin{align}
\beta_{o}&=\frac{m_{inf}}{M}\\
\beta_s&=\frac{1}{M}\\
\beta_e&= \frac{G_{bg} + g^{occ}_{\omega} N}{M}
\end{align}

Then we have

\begin{align}
\dot\omega_z =
- (\beta_o+m_{sa}\beta_s) \omega_z
\ +\ \beta_o \omega_o + m_{sa}\beta_{s} \omega_{sa}
 + \beta_e
\end{align}

***

### **Zone $CO_2$ Concentration**
\begin{align}
M\dot c_z =
- (m_{inf} + m_{sa}) c_z
\ +\ m_{inf} c_o + m_{sa} c_{sa}
 + g_{CO2}^{occ} N
\end{align}
where:
- $c_o,\, c_z,\, c_{sa}$ = outdoor, zone, supply air $CO_2$ concentrations $[kg/kg_{dry}]$
- $g^{occ}_{CO2}$ = per-person $CO_2$ generation rate $[kg/s]$.

And we also have

\begin{align}
\dot{c}_z &=
- (\beta_o+m_{sa}\beta_s) c_z
\ +\ \beta_o c_o + m_{sa}\beta_{s} c_{sa}
 + \gamma_e
\end{align}

where
\begin{align}
\gamma_e&= \frac{g_{CO2}^{occ} N}{M}
\end{align}

---

### Dynamic Equations

\begin{align}
\dot T_z &=
- (\alpha_o+m_{sa}\alpha_s)T_z+\ \alpha_o T_o + m_{sa}\alpha_s T_{sa}
+ \alpha_e\\
\dot\omega_z &=
- (\beta_o+m_{sa}\beta_s) \omega_z
\ +\ \beta_o \omega_o + m_{sa}\beta_{s} \omega_{sa}
 + \beta_e\\
\dot{c}_z &=
- (\beta_o+m_{sa}\beta_s) c_z
\ +\ \beta_o c_o + m_{sa}\beta_{s} c_{sa}
 + \gamma_e
\end{align}

***

These forms are **physically complete, transparent, and directly express the conservation of energy, mass (moisture), and tracer (CO₂) for an air-conditioned zone** with standard HVAC inputs.


---

### **Steady state equations**

\begin{align}
T_z^\star
&=\frac{UA\,T_o\;+\;c_{pa}\,m_{\inf}\,T_o\;+\;c_{pa}\,m_{sa}\,T_{sa}\;+\;Q_{bg}\;+\;f_c\,q^{occ}_{sens}N}
{UA\;+\;c_{pa}\,(m_{sa}+m_{\inf})}\\
\omega_z^\star
&=\frac{m_{\inf}\,\omega_o\;+\;m_{sa}\,\omega_{sa}\;+\;G_{bg}\;+\;g^{occ}_{\omega}N}
{m_{sa}+m_{\inf}}\\
c_z^\star
&=\frac{m_{\inf}\,c_o\;+\;m_{sa}\,c_{sa}\;+\;g^{occ}_{CO2}N}
{m_{sa}+m_{\inf}}
\end{align}

Written in terms of the transformed coefficients:
\begin{align}
T^\star_z &=
\ \frac{\alpha_o}{(\alpha_o+m_{sa}\alpha_s)} T_o + \frac{m_{sa}\alpha_s}{(\alpha_o+m_{sa}\alpha_s)} T_{sa}
+ \frac{\alpha_e}{(\alpha_o+m_{sa}\alpha_s)}\\
\omega^\star_z &=
\ \frac{\beta_o}{(\beta_o+m_{sa}\beta_s)} \omega_o + \frac{{m_{sa}}\beta_{s}}{(\beta_o+m_{sa}\beta_s)} \omega_{sa}
 + \frac{\beta_e}{(\beta_o+m_{sa}\beta_s)}\\
{c}^\star_z &=
\frac{\beta_o}{(\beta_o+m_{sa}\beta_s)} c_o + \frac{m_{sa}\beta_{s}}{(\beta_o+m_{sa}\beta_s)} c_{sa}
 + \frac{\gamma_e}{(\beta_o+m_{sa}\beta_s)}
\end{align}



## Paramater Estimation

### **Uncertainty modeling and estimation**

We will assume that $\{T_o, \omega_o, c_o, T_{sa},\omega_{sa},c_{sa}\}$ are measured accurately.

We will consider the augmented state
\begin{align}
x_k &\triangleq
\begin{bmatrix}\alpha_{o,k} & \alpha_{s,k} & \alpha_{e,k} & \beta_{o,k} & \beta_{s,k} & \beta_{e,k} & \gamma_{e,k}
& T_{z,k} & \omega_{z,k} & c_{z,k}
\end{bmatrix}^T
\end{align}

The discrete time evolution of the system is then:
\begin{align}
x_k & = f(x_{k-1})+w_k, \qquad w_k \sim \mathscr{N}(0,\Sigma_Q),\\
y_k & = H_k x_k + \varepsilon_k, \qquad \varepsilon_k \sim \mathscr{N}(0,\Sigma_R)
\end{align}
where
\begin{align}
f({x}_{k-1}) = {x}_{k-1}+\Delta t\begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ -({\alpha}_{o,k-1}+m_{sa,k-1}{\alpha}_{s,k-1}){T}_{z,k-1} + {\alpha}_{o,k-1}T_{o,k-1} + m_{sa,k-1}{\alpha}_{s,k-1}T_{sa,k-1} + {\alpha}_{e,k-1}\\
-({\beta}_{o,k-1}+m_{sa,k-1}{\beta}_{s,k-1}){\omega}_{z,k-1} + {\beta}_{o,k-1}\omega_{o,k-1} + m_{sa,k-1}{\beta}_{s,k-1}\omega_{sa,k-1} + {\beta}_{e,k-1}\\
-({\beta}_{o,k-1}+m_{sa,k-1}{\beta}_{s,k-1}){c}_{z,k-1} + {\beta}_{o,k-1|k-1}c_{o,k-1} + m_{sa,k-1}{\beta}_{s,k-1}c_{sa,k-1} + {\gamma}_{e,k-1} \end{bmatrix}\\
H_k & =
\begin{bmatrix}0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\
\end{bmatrix}\\
y_k &= \begin{bmatrix}
T_{z,k}\\ \omega_{z,k} \\ c_{z,k}
\end{bmatrix}
\end{align}

---

#### **The EKF**

1. Prediction Step
\begin{align}
\widehat{x}_{k|k-1}=f(\widehat{x}_{k-1|k-1})
\end{align}


2. Predict the error covariance

\begin{align}
P_{k|k-1}=F_{k-1}P_{k-1|k-1}F_{k-1}^{T}+\Sigma _{Q}
\end{align}

3. Compute the Kalman gain

\begin{align}
K_{k}=P_{k|k-1}H_{k}^{T}(H_{k}P_{k|k-1}H_{k}^{T}+\Sigma _{R})^{-1}
\end{align}

4. Update the state estimate
\begin{align}
\widehat{x}_{k|k}=\widehat{x}_{k|k-1}+K_{k}(y_{k}-H_{k}\widehat{x}_{k|k-1})
\end{align}

5. Update the error covariance
\begin{align}
P_{k|k}=(I-K_{k}H_{k})P_{k|k-1}
\end{align}


Here
\begin{align}
f(\widehat{x}_{k-1|k-1}) \triangleq \widehat{x}_{k-1|k-1}+\Delta t\begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ -(\widehat{\alpha}_{o,k-1|k-1}+m_{sa,k-1}\widehat{\alpha}_{s,k-1|k-1})\widehat{T}_{z,k-1|k-1} + \widehat{\alpha}_{o,k-1|k-1}T_{o,k-1} + m_{sa,k-1}\widehat{\alpha}_{s,k-1|k-1}T_{sa,k-1} + \widehat{\alpha}_{e,k-1|k-1}\\
-(\widehat{\beta}_{o,k-1|k-1}+m_{sa,k-1}\widehat{\beta}_{s,k-1|k-1})\widehat{\omega}_{z,k-1|k-1} + \widehat{\beta}_{o,k-1|k-1}\omega_{o,k-1} + m_{sa,k-1}\widehat{\beta}_{s,k-1|k-1}\omega_{sa,k-1} + \widehat{\beta}_{e,k-1|k-1}\\
-(\widehat{\beta}_{o,k-1|k-1}+m_{sa,k-1}\widehat{\beta}_{s,k-1|k-1})\widehat{c}_{z,k-1|k-1} + \widehat{\beta}_{o,k-1|k-1}c_{o,k-1} + m_{sa,k-1}\widehat{\beta}_{s,k-1|k-1}c_{sa,k-1} + \widehat{\gamma}_{e,k-1|k-1} \end{bmatrix}
\end{align}
and the Jacobian at $\widehat{x}_{k-1|k-1}$ is
\begin{align}
F_{k-1} &= I+ \Delta t\begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
-\widehat{T}_{z,k-1|k-1}+T_{o,k-1} & -m_{sa,k-1}\widehat{T}_{z,k-1|k-1}+m_{sa,k-1}T_{sa,k-1} & 1 & 0 & 0 & 0 & 0 & -(\widehat{\alpha}_{o,k-1|k-1}+m_{sa,k-1}\widehat{\alpha}_{s,k-1|k-1}) & 0 & 0 \\
0 & 0 & 0 & -\widehat{\omega}_{z,k-1|k-1}+\omega_{o,k-1} & -m_{sa,k-1}\widehat{\omega}_{z,k-1|k-1}+m_{sa,k-1}\omega_{sa,k-1} & 1 & 0 & 0 & -(\widehat{\beta}_{o,k-1|k-1}+m_{sa,k-1}\widehat{\beta}_{s,k-1|k-1}) & 0 \\
0 & 0 & 0 & -\widehat{c}_{z,k-1|k-1}+c_{o,k-1} & -m_{sa,k-1}\widehat{c}_{z,k-1|k-1}+m_{sa,k-1}c_{sa,k-1} & 0 & 1 & 0 & 0 & -(\widehat{\beta}_{o,k-1|k-1}+m_{sa,k-1}\widehat{\beta}_{s,k-1|k-1}) \end{bmatrix}
\end{align}

##HVAC Control

Here are generalized expressions ** for an HVAC system with **air mixing, cooling/heating coil, and reheat**:

***

### **Air Mixing (Pre-coil)**
Assume the air handler mixes **outdoor air (OA)** and **return air (RA)**:
- $ m_{oa,k} $: outdoor air mass flow
- $ m_{ra,k} $: return air mass flow
- $ m_{ma,k} = m_{oa,k} +m_{ra,k}$: total mixed air mass flow

The **mixed air properties** entering the coil are:

\begin{align}
T_{ma,k} &= \frac{m_{oa,k} T_{o,k} + m_{ra,k} T^\star_{z,k}}{m_{ma,k}},\\
\omega_{ma,k} &= \frac{m_{oa,k}\omega_{o,k} + m_{ra,k}\omega^\star_{z,k}}{m_{ma,k}},\\
c_{m,k} &= \frac{m_{oa,k}c_{o,k} + m_{ra,k}c^\star_{z,k}}{m_{ma,k}}
\end{align}

Let $\gamma_{m,k}=\frac{m_{o,k}}{m_{oa,k}+m_{ra,k}}$ then
\begin{align}
T_{ma,k} &= m_{ma,k}\left(\gamma_{m,k} T_{o,k} + (1-\gamma_{m,k}) T^\star_{z,k}\right),\\
\omega_{ma,k} &= m_{ma,k}\left(\gamma_{m,k}\omega_{o,k} + (1-\gamma_{m,k})\omega^\star_{z,k}\right),\\
c_{ma,k} &= m_{ma,k}\left(\gamma_{m,k}c_{o,k} + (1-\gamma_{m,k})c^\star_{z,k}\right)
\end{align}

---



### Sensible (Thermal) Energy Required by the Coil

For each time step, the energy rate required to move the mixed air from **mixed condition ($T_{ma,k}$)** to the **coil outlet or supply air temperature (${T}_{sa,k+1}$)** is:
\begin{align}
Q_{\text{coil},k+1} = {m}_{ma,k} \, c_{pa} |({T}_{\text{sa},k+1} - T_{\text{ma},k})|
\end{align}
- ${T}_{\text{sa},k+1}$: supply air temperature [°C] for the next step
- $T_{\text{ma},k}$: mixed air temperature entering coil [°C]

***

### Latent (Moisture) Energy Cost

If moisture is **removed** (dehumidification by cooling coil):
\begin{align}
Q_{\text{latent},k} &= {m}_{ma,k} \cdot h_{fg} \cdot \max(0, \omega_{\text{ma},k} - {\omega}^\star_{z,k+1})
\end{align}

- $h_{fg}$: latent heat of vaporization (≈ 2,500,000 J/kg at room temperature)
- $\omega_{\text{ma},k}$: mixed air humidity ratio (kg water/kg dry air)
- ${\omega}_{\text{sa},k+1}$: supply air humidity ratio at the next step

(Coils don't **add** moisture, so this term is only positive when $\omega_{sa,k} < \omega_{ma,k+1}$.)

***

### Combined Coil Power (Cooling or Heating)

The **total coil load at time $k$** (assuming no reheat for simplicity) is:
\begin{align}
Q_{\text{coil,total},k+1} &= Q_{\text{coil},k+1} + Q_{\text{latent},k+1},\\
&= {m}_{ma,k} \left[c_{pa}|({T}_{\text{sa},k+1} - T_{\text{ma},k})| + h_{fg} \cdot \max(0, \omega_{\text{ma},k} - {\omega}_{\text{sa},k+1}) \right]\\
&= {m}_{ma,k} \left[c_{pa}|\left({T}_{\text{sa},k+1} - \left(\gamma_{m,k} T_{o,k} + (1-\gamma_{m,k}) T^\star_{z,k}\right)\right)| + h_{fg} \cdot \max\left(0, \left(\gamma_{m,k}\omega_{o,k} + (1-\gamma_{m,k})\omega^\star_{z,k}\right) - {\omega}_{\text{sa},k+1}\right) \right]
\end{align}

Substituting the Kalman filter estimate $\mu_k$
\begin{align}
\widehat{Q}_{\text{coil,total},k+1} &= {m}_{ma,k} \left[c_{pa}|\left({T}_{\text{sa},k+1} - \left(\gamma_{m,k} T_{o,k} + (1-\gamma_{m,k}) \widehat{T}^\star_{z,k}\right)\right)| + h_{fg} \cdot \max\left(0, \left(\gamma_{m,k}\omega_{o,k} + (1-\gamma_{m,k})\widehat{\omega}^\star_{z,k}\right) - {\omega}_{\text{sa},k+1}\right) \right]
\end{align}

The optimal control HVAC problem is to then find $(T_{sa,k+1},{\omega}_{{sa},k+1})\in D$ and $\gamma_{m,k}\in [0,1]$ such that $\widehat{Q}_{\text{coil,total},k+1}$ is minimized.

***

### Supply mass flowrate

Predicted zone temperature and moisture at the next time step.


\begin{align}
\widehat{T}^\star_{z,k+1} &\approx
 \frac{\widehat{\alpha}_{o,k}}{(\widehat{\alpha}_{o,k}+m_{sa,k}\widehat{\alpha}_{s,k})} T_{o,k+1} + \frac{m_{sa,k}\widehat{\alpha}_{s,k}}{(\widehat{\alpha}_{o,k}+m_{sa,k}\widehat{\alpha}_{s,k})} T_{sa,k+1}
+\frac{\widehat{\alpha}_{e,k}}{(\widehat{\alpha}_{o,k}+m_{sa,k}\widehat{\alpha}_{s,k})}\\
\widehat{\omega}^\star_{z,k+1} &\approx
\ \frac{\widehat{\beta}_{o,k}}{(\widehat{\beta}_{o,k}+m_{sa,k}\widehat{\beta}_{s,k})} \omega_{o,k+1} + \frac{{m_{sa,k}}\widehat{\beta}_{s,k}}{(\widehat{\beta}_{o,k}+m_{sa,k}\widehat{\beta}_{s,k})} \omega_{sa,k+1}
+\frac{\widehat{\beta}_{e,k}}{(\widehat{\beta}_{o,k}+m_{sa,k}\widehat{\beta}_{s,k})}\\
\widehat{c}^\star_{z,k+1} &\approx
\frac{\widehat{\beta}_{o,k}}{(\widehat{\beta}_{o,k}+m_{sa,k}\widehat{\beta}_{s,k})} c_{o,k+1} + \frac{m_{sa,k}\widehat{\beta}_{s,k}}{(\widehat{\beta}_{o,k}+m_{sa,k}\widehat{\beta}{s,k})} c_{sa,k+1}
+\frac{\widehat{\gamma}_{e,k}}{(\widehat{\beta}_{o,k}+m_{sa,k}\widehat{\beta}_{s,k})}
\end{align}

* Check if $(T^\star_{z,k+1},\omega^\star_{z,k+1})\in D$
* If not increase $m_{sa,k+1}$

---

### Distributed Control of Multi-zone HVAC Systems Considering Indoor Air Quality

https://arxiv.org/pdf/2003.08208

# Sri Lanka Weather Data

https://climate.onebuilding.org/WMO_Region_2_Asia/LKA_Sri_Lanka/index.html

#Energy Plus Simulation

##Install

In [ ]:
# install from dev branch
!pip install -q "energy-plus-utility @ git+https://github.com/mugalan/energy-plus-utility.git@dev"

In [ ]:
# run the silent bootstrap in this kernel
from eplus import prepare_colab_eplus
prepare_colab_eplus()  # raises on failure, otherwise silent
from eplus import EPlusUtil, EPlusSqlExplorer

## Initialize class

In [ ]:
import subprocess, json, pathlib, os
import pandas as pd
EPLUS = str(pathlib.Path.home() / "EnergyPlus-25-1-0")
EPLUS_ROOT = "/root/EnergyPlus-25-1-0"

out_dir = "/content/eplus_out"
idf = f"{EPLUS}/ExampleFiles/5ZoneAirCooled.idf"
epw = f"{EPLUS}/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"

util = EPlusUtil(verbose=1, out_dir = out_dir)
util.delete_out_dir()
util.set_model(idf,epw, outdoor_co2_ppm=400.0, per_person_m3ps_per_W=3.82e-08)
util.ensure_output_sqlite()
util.enable_runtime_logging()

In [ ]:
def test_method (self,s,aa):
    print('inside test',aa)
    return aa

In [ ]:
import types
util.test_method = types.MethodType(test_method, util)

In [ ]:
util.test_method(0,1)

## *Optional:* Convert the model to .json

In [ ]:
idf_path = pathlib.Path(idf)
converter = os.path.join(EPLUS_ROOT, "ConvertInputFormat")   # on Windows it's ConvertInputFormat.exe

# Convert IDF → epJSON (outputs 5ZoneAirCooled.epJSON in the same folder)
subprocess.run([converter, str(idf_path)], check=True)

epjson_path = idf_path.with_suffix(".epJSON")
print("epJSON exists?", epjson_path.exists(), epjson_path)

## Simulations

###Dry run to create tables etc.

In [ ]:
util = EPlusUtil(verbose=1)
util.delete_out_dir()
util.set_model(idf,epw)
util.ensure_output_sqlite()
util.dry_run_min(include_ems_edd=False)

In [ ]:
util.list_zone_names()

In [ ]:
util.list_available_variables()

In [ ]:
actuators_df=util.list_available_actuators()
actuators_df

In [ ]:
# Assuming you have a DataFrame named 'df' with a column named 'ColumnName'
# Filter rows where 'ColumnName' contains 'VAV'
filtered_df = actuators_df[actuators_df['ActuatorKey'].str.contains('VAV', na=False)]

# Display the filtered DataFrame
display(filtered_df)

In [ ]:
filtered_df.to_dict(orient='records')

In [ ]:
util.list_available_meters()

### Run Simulation

In [ ]:

# util.ensure_output_variables([
#     {"name":"Zone Air System Sensible Cooling Energy", "key":"*", "freq":"TimeStep"},
#     {"name":"Zone Total Internal Latent Gain Energy", "key":"*", "freq":"TimeStep"},
#     {"name": "Zone Air CO2 Concentration", "key": "*", "freq": "TimeStep"},
#     {"name": "Zone Outdoor Air Inlet Mass Flow Rate", "key": "*", "freq": "TimeStep"},
#     {"name": "System Node Standard Density Volume Flow Rate", "key": "*", "freq": "TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE2-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE3-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE4-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE5-1", "freq":"TimeStep"},
# ], activate=True)
# 1) Make sure SQL will be produced
# util.ensure_output_sqlite(activate=True)

# site_additional_vars = [
#     "Site Wind Speed",
#     "Site Wind Direction",
#     "Site Diffuse Solar Radiation Rate per Area",
#     "Site Direct Solar Radiation Rate per Area",
#     "Site Horizontal Infrared Radiation Rate per Area",
#     "Site Sky Temperature",
# ]
# site_additional_specs=[{'name': v, 'key': 'Environment', 'freq': 'TimeStep'} for v in site_additional_vars]

In [ ]:
specs = [
    # --- Zone state + people ---
    {"name": "Zone Mean Air Temperature",                "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Mean Air Dewpoint Temperature",       "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Air Relative Humidity",               "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Mean Air Humidity Ratio",             "key": "*",            "freq": "TimeStep"},
    {"name": "Zone People Occupant Count",               "key": "*",            "freq": "TimeStep"},

    # --- CO₂ & OA into zones ---
    {"name": "Zone Air CO2 Concentration",               "key": "*",            "freq": "TimeStep"},

    # --- Site weather (Environment key) ---
    {"name": "Site Outdoor Air Drybulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Wetbulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Dewpoint Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Relative Humidity",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Humidity Ratio",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Barometric Pressure",        "key": "Environment", "freq": "TimeStep"},
    {"name": "Site Outdoor Air CO2 Concentration",                          "key": "Environment",  "freq": "TimeStep"},

    {"name": "System Node Temperature",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node Mass Flow Rate",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node Humidity Ratio",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node CO2 Concentration",               "key": "*",            "freq": "TimeStep"},
]

# 1) Ensure the Output:Variable objects exist (dedup-aware)
util.ensure_output_variables(specs, activate=True)


# 2) Ensure the meter(s) you want are reported
output_meters = ["InteriorLights:Electricity:Zone:SPACE5-1","Cooling:EnergyTransfer:Zone:SPACE1-1","Cooling:EnergyTransfer","Electricity:Facility","ElectricityPurchased:Facility", "ElectricitySurplusSold:Facility"]
util.ensure_output_meters(output_meters, freq="TimeStep")



#3) Register callbacks
# util.register_handlers(
#     "after_hvac",
#     [{"method_name": "probe_zone_air_and_supply",
#       "key_wargs": {"log_every_minutes": 1, "precision": 3}}],
#     clear=False, run_during_warmup=False
# )

util.register_handlers(
    "after_hvac",
    [{"method_name": "occupancy_handler","key_wargs": {"lam": 33.0, "min": 20, "max": 45, "seed": 123}}],
    clear=False, run_during_warmup=False
)

# util.register_handlers(
#     "begin",
#     [{"method_name": "probe_zone_air_and_supply_with_kf",
#      "key_wargs": {
#          "log_every_minutes": 15,
#          "precision": 3,

#          "kf_db_filename": "eplusout_kf_test.sqlite",
#          "kf_batch_size": 50,
#          "kf_commit_every_batches": 10,
#          "kf_checkpoint_every_commits": 5,
#          "kf_journal_mode": "WAL",
#          "kf_synchronous": "NORMAL",

#          # --- 10-state init (αo, αs, αe, βo, βs, βe, γe, Tz, wz, cz)
#          "kf_init_mu":        [0.1, 0.1, 0.0,  0.1, 0.1, 0.0,  0.0,  20.0, 0.008, 400.0],
#          "kf_init_cov_diag":  [1.0, 1.0, 1.0,  1.0, 1.0, 1.0,  1.0,  25.0, 1e-3,  1e3  ],
#          "kf_sigma_P_diag":   [1e-6,1e-6,1e-6, 1e-6,1e-6,1e-6, 1e-6, 1e-5, 1e-6,  1e-4 ],

#          # Optional: pretty column names for state persistence (dynamic schema)
#          "kf_state_col_names": [
#              "alpha_o","alpha_s","alpha_e","beta_o","beta_s","beta_e","gamma_e","Tz","wz","cz"
#          ],

#          # Use the 10-state EKF preparer
#          "kf_prepare_fn": util._kf_prepare_inputs_zone_energy_model
#      }}],
#     clear=True
# )
# util.register_handlers(
#     "before_hvac",
#     [{"method_name": "tick_set_actuator",
#       "kwargs": {
#         "component_type": "System Node Setpoint",
#         "control_type":   "Mass Flow Rate Setpoint",
#         "actuator_key":   "SPACE4-1 ZONE COIL AIR IN NODE",
#         "value":          0.35,            # kg/s request
#         "when":           "success",
#         "read_back":      True,            # read back actuator value after setting
#         "precision":      4
#       }}],
#     run_during_warmup=False
# )
# util.register_handlers(
#     "begin",   # or "after_hvac", etc.
#     [{"method_name": "tick_set_actuator",
#       "kwargs": {
#         "component_type": "People",
#         "control_type": "Number of People",
#         "actuator_key": "SPACE1-1 PEOPLE 1",
#         "value": 22.0,
#         "when": "success",
#         "read_back": True,
#         "precision": 3
#       }}],
#     run_during_warmup=False
# )
# util.register_handlers(
#     "after_hvac",
#     [{"method_name": "tick_log_actuator",
#       "kwargs": {
#         "component_type": "System Node Setpoint",
#         "control_type":   "Mass Flow Rate Setpoint",
#         "actuator_key":   "SPACE4-1 ZONE COIL AIR IN NODE",
#         "when": "always", #"on_change",
#         "precision": 3
#       }}],
#     run_during_warmup=False
# )
util.register_handlers(
    "after_hvac",   # alias for callback_begin_system_timestep_before_predictor
    [{
        "method_name": "tick_log_meter",
        "kwargs": {
            "name": "Electricity:Facility",
            "which": "value",            # current tick value
            "when": "always",         # only log when it changes
            "precision": 3,
            "include_timestamp": True,
            "allow_warmup": False        # skip during sizing/warmup
        }
    }],
    clear=False,
    run_during_warmup=False
)
# util.register_handlers(
#     "begin",  # callback_begin_system_timestep_before_predictor
#     [{
#         "method_name": "tick_log_variable",
#         "kwargs": {
#             "name": "Zone People Occupant Count",
#             "key": "SPACE1-1",          # <-- replace with your zone name
#             "when": "always",        # log only when it changes
#             "precision": 0,             # people → integers are nice to see
#             "include_timestamp": True,
#             "allow_warmup": False
#         }
#     }],
#     clear=False,
#     run_during_warmup=False
# )
rc=util.run_annual()

### Variables/Actuators Tables

In [ ]:
util.list_available_variables()

In [ ]:
util.list_available_actuators() #.to_dict(orient='records')

In [ ]:
util.list_available_meters()

### SQL Table Inspect

In [ ]:
xp = EPlusSqlExplorer(f"{out_dir}/eplusout.sql")

In [ ]:
xp.list_tables()

In [ ]:
df = xp.list_sql_variables(name="System Node Temperature")
df[['KeyValue','n_rows']].head(20)

### Analyze weather data

In [ ]:
util.export_weather_sql_to_csv()

In [ ]:
weather_df=pd.read_csv('eplus_out/weather_timeseries.csv')
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
weather_df['month'] = weather_df['timestamp'].dt.month

In [ ]:
weather_df

In [ ]:
import numpy as np
from scipy.stats import norm, lognorm, gamma
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Extract the temperature data
variable='Site Outdoor Air Humidity Ratio [kgWater/kgDryAir]'#'Site Outdoor Air Dewpoint Temperature [C]' #'Site Outdoor Air Drybulb Temperature [C]' #
n=9
data_df = weather_df[weather_df['month']==n]
data = data_df[variable]

# Fit a normal distribution to the data:
mu, std = norm.fit(data)

# Create the histogram trace from the previous plot
counts, bin_edges = np.histogram(data, bins=50) # Adjust bin count as needed
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

histogram_trace = go.Bar(x=bin_centers, y=counts, name='Histogram', opacity=0.7)

# Create the Gaussian curve trace
xmin, xmax = data.min(), data.max()
x_norm = np.linspace(xmin, xmax, 100)
p_norm = norm.pdf(x_norm, mu, std)

# Scale the PDF to match the histogram's count scale
bin_width = bin_edges[1] - bin_edges[0]
scaled_pdf_norm = p_norm * len(data) * bin_width

gaussian_trace = go.Scatter(x=x_norm, y=scaled_pdf_norm, mode='lines', name=f'Gaussian Fit (μ={mu:.2f}, σ={std:.2f})', line=dict(color='red', width=2))

# Fit Log-Normal distribution
# Log-normal distribution requires positive data. Since temperature can be negative,
# a simple log-normal fit might not be appropriate directly.
# However, for demonstration, we can fit it to the positive part or shift the data.
# Let's fit it to the original data, understanding the limitations if negative values exist.
# We need to be careful if temperature_data contains zero or negative values for lognorm fit.
# For simplicity, we'll add an offset if there are non-positive values.
offset = 0
if (data <= 0).any():
    offset = -data.min() + 1 # Shift data to be positive
    print(f"Shifting data by {offset:.2f} for Log-Normal fit to ensure positivity.")

shape_lognorm, loc_lognorm, scale_lognorm = lognorm.fit(data + offset)

# Generate points for the fitted Log-Normal curve
# Ensure the x range is appropriate for the shifted data
x_lognorm = np.linspace(data.min() + offset, data.max() + offset, 100)
p_lognorm = lognorm.pdf(x_lognorm, shape_lognorm, loc_lognorm, scale_lognorm)

# Scale the PDF and shift x back for plotting
scaled_pdf_lognorm = p_lognorm * len(data) * bin_width
x_lognorm_unshifted = x_lognorm - offset

lognormal_trace = go.Scatter(x=x_lognorm_unshifted, y=scaled_pdf_lognorm, mode='lines', name=f'Log-Normal Fit', line=dict(color='green', width=2))


# Fit Gamma distribution
# Gamma distribution also typically requires positive data. Similar consideration as Log-Normal.
# We'll fit it to the shifted data if an offset was applied for lognormal.
shape_gamma, loc_gamma, scale_gamma = gamma.fit(data + offset)

# Generate points for the fitted Gamma curve
# Ensure the x range is appropriate for the shifted data
x_gamma = np.linspace(data.min() + offset, data.max() + offset, 100)
p_gamma = gamma.pdf(x_gamma, shape_gamma, loc_gamma, scale_gamma)

# Scale the PDF and shift x back for plotting
scaled_pdf_gamma = p_gamma * len(data) * bin_width
x_gamma_unshifted = x_gamma - offset


gamma_trace = go.Scatter(x=x_gamma_unshifted, y=scaled_pdf_gamma, mode='lines', name=f'Gamma Fit', line=dict(color='purple', width=2))


# Create the figure and add traces
fig = go.Figure()
fig.add_trace(histogram_trace)
fig.add_trace(gaussian_trace)
fig.add_trace(lognormal_trace)
fig.add_trace(gamma_trace)


# Update layout
fig.update_layout(title=f'Distribution of {variable} with Distribution Fits',
                  xaxis_title=variable,
                  yaxis_title='Count',
                  barmode='overlay' # Overlay bars to see fits better
                 )

# Show the plot
fig.show()

### Plot Results

#### Zone plots

In [ ]:
# Drybulb (auto-picks top zone keys if keys=None)
fig1=util.plot_sql_zone_variable(
    "Zone Mean Air Temperature",
    keys=["*"],                          # auto-pick a few zones with data
    reporting_freq=("TimeStep",),       # match how you logged
    resample="1h",
    title="Zone Mean Air Temperature"
)

# Humidity ratio
fig2=util.plot_sql_zone_variable(
    "Zone Mean Air Humidity Ratio",
    keys=["*"],
    reporting_freq=("TimeStep",),
    resample="1h",
    title="Zone Mean Air Humidity Ratio"
)

# CO2 concentration
fig3=util.plot_sql_zone_variable(
    "Zone Air CO2 Concentration",
    keys={"*"},
    reporting_freq=("TimeStep",),
    resample="1h",
    title="Zone Air CO2 Concentration"
)

#### Outdoor Air Plots

In [ ]:
sels = [
    {"kind":"var", "name":"Site Outdoor Air Drybulb Temperature", "key":"Environment", "label":"Tdb [C]"},
    {"kind":"var", "name":"Site Outdoor Air Dewpoint Temperature", "key":"Environment", "label":"Tdew [C]"},
    {"kind":"var", "name":"Site Outdoor Air Humidity Ratio",      "key":"Environment", "label":"w [kg/kg]"},
]
fig4=util.plot_sql_series(
    selections=sels,
    reporting_freq=("TimeStep",),
    include_design_days=False,
    resample="1h",
    meters_to_kwh=False,
    title="Outdoor (Environment)"
)

#### Supply Air Plots

In [ ]:
z2nodes = util._discover_zone_inlet_nodes_from_sql()
zone = "SPACE4-1"
sels = [{"kind":"var", "name":"System Node Mass Flow Rate", "key":n, "label":n} for n in z2nodes[zone]]
fig5=util.plot_sql_series(
    selections=sels,
    reporting_freq=("TimeStep",),
    resample="15min",
    meters_to_kwh=False,
    title=f"{zone} — Supply Node Mass Flow Rate"
)

In [ ]:
sels = [{"kind":"var", "name":"System Node Temperature", "key":n, "label":n} for n in z2nodes[zone]]
fig6=util.plot_sql_series(selections=sels, reporting_freq=("TimeStep",), resample="15min",
                     meters_to_kwh=False, title=f"{zone} — Supply Node Temperature")

In [ ]:
sels = [{"kind":"var", "name":"System Node Humidity Ratio", "key":n, "label":n} for n in z2nodes[zone]]
fig7=util.plot_sql_series(selections=sels, reporting_freq=("TimeStep",), resample="15min",
                     meters_to_kwh=False, title=f"{zone} — Supply Node Humidity Ratio")

In [ ]:
sels = [{"kind":"var", "name":"System Node CO2 Concentration", "key":n, "label":n} for n in z2nodes[zone]]
fig8=util.plot_sql_series(selections=sels, reporting_freq=("TimeStep",), resample="15min",
                     meters_to_kwh=False, title=f"{zone} — Supply Node CO2 Concentration")

#### Occupant plot

In [ ]:
# 1) discover zone keys that exist for the variable
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,            # don't filter; accept Zone Timestep, Hourly, etc.
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)

# (optional) limit to first N zones
# occ_keys = occ_keys[:8]

# 2) build selections and plot
selections = [
    {'kind':'var', 'name':'Zone People Occupant Count', 'key':k, 'label':k}
    for k in occ_keys
]

fig = util.plot_sql_series(
    selections=selections,
    reporting_freq=None,      # pull whatever is in the DB
    resample='1h',            # average to hourly; set to None for native timestep
    aggregate_vars='mean',    # hourly mean occupancy; use 'sum' for person-hours per hour
    title='Occupant Count per Zone',
    show=True
)

#### Covariance plots

In [ ]:
# 1) discover the exact zone keys present for the occupancy variable
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,              # don't filter; show all
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)

# 2) build selections using those keys (so keys and zones match the DB)
# output_sels = [
#     {'kind':'var','name':'Zone Mean Air Temperature','key':k,'label':f'MAT: {k}'}
#     for k in occ_keys
# ] + [
#     {'kind':'var','name':'Zone Air Relative Humidity','key':k,'label':f'ARH: {k}'}
#     for k in occ_keys
# ] +
# output_sels =  [
#     {'kind':'var','name':'Zone Air System Sensible Cooling Energy','key':k,'label':f'QSEN: {k}'}
#     for k in occ_keys
# ]
# + [
#     {'kind':'var','name':'Zone Total Internal Latent Gain Energy','key':k,'label':f'QLAT: {k}'}
#     for k in occ_keys
# ]
output_sels = [
    {'kind':'var','name':'Zone Air CO2 Concentration','key':k,'label':f'CO2: {k}'}
    for k in occ_keys
]

control_sels = (
    [{'kind':'var','name':'Zone People Occupant Count','key':k,'label':f'Occ: {k}'} for k in occ_keys]
    # + [
    #     {'kind':'var','name':'Site Outdoor Air Drybulb Temperature','key':'Environment','label':'OAT'},
    #     {'kind':'var','name':'Site Outdoor Air Wetbulb Temperature','key':'Environment','label':'OWB'},
    # ]
)

# 3) plot covariance (pull any freq; we resample to 1H anyway)
fig = util.plot_sql_cov_heatmap(
    control_sels=control_sels,
    output_sels=output_sels,
    reporting_freq=None,     # <- don't filter out Zone Timestep rows
    resample='1h',           # compute cov on hourly series
    reduce='mean',
    stat='cov',              # or 'corr' if you want scale-free
    min_periods=12,
    include_design_days=False
)

#### Kalman Filter estimates plots

In [ ]:
xp=EPlusSqlExplorer(sql_path="eplus_out/eplusout_kf_test.sqlite")

In [ ]:
# If you used the suggested test DB/table names:
df = xp.get_table_data(db="eplus_out/eplusout_kf_test.sqlite", table="KalmanEstimates")
zone1_df=df[df["Zone"]=="SPACE1-1"]

In [ ]:
df

In [ ]:

zone1_df

In [ ]:
import plotly.graph_objects as go

def plot_df_columns(df, x_column, trace_columns):
    """
    Generates a plotly line plot for specified columns in a DataFrame.

    Args:
        df: pandas DataFrame
        x_column: Name of the column to use for the x-axis.
        trace_columns: A list of column names to plot as separate traces on the y-axis.

    Returns:
        A plotly Figure object.
    """
    fig = go.Figure()

    for col in trace_columns:
        if col in df.columns:
            fig.add_trace(go.Scattergl(x=df[x_column], y=df[col], mode='lines', name=col))
        else:
            print(f"Warning: Column '{col}' not found in DataFrame.")

    fig.update_layout(
        title="Line Plot of DataFrame Columns",
        xaxis_title=x_column,
        yaxis_title="Value"
    )

    return fig

# Example usage with your 'zone1_df'
# Make sure 'Timestamp' is a datetime type for proper plotting
if 'Timestamp' in zone1_df.columns:
    zone1_df['Timestamp'] = pd.to_datetime(zone1_df['Timestamp'])

# Define the columns to plot
x_col = 'Timestamp'
y_cols = ['y_T', 'yhat_T', 'y_w', 'yhat_w', 'y_c', 'yhat_c']
# y_cols = ['mu_0', 'mu_1','mu_2', 'mu_3', 'mu_4', 'mu_5', 'mu_6']

# Generate and show the plot
fig = plot_df_columns(zone1_df, x_col, y_cols)
fig.show()

#### Other plots

In [ ]:
variable="System Node Mass Flow Rate"  #"System Node Temperature" #"Site Outdoor Air Barometric Pressure"#"Zone Inlet Air CO2 Concentration"#"Site Outdoor Air Relative Humidity" #"Zone Air Relative Humidity" #"Zone Outdoor Air Inlet Mass Flow Rate" #"Zone Infiltration Standard Density Volume Flow Rate" #'Site Outdoor Air Wetbulb Temperature' #"Zone Mean Air Temperature" #"Zone Air Relative Humidity" #"Zone Air CO2 Concentration" #"System Node Mass Flow Rate" #"Zone Air CO2 Concentration" #"Zone Air Relative Humidity" #"Zone Air Temperature"
# 4) discover keys and plot
display(util.list_sql_zone_variables(name=variable).head(10))

In [ ]:
zone_fig=util.plot_sql_zone_variable(
    variable,
    keys=["*"], #["SPACE1-1","SPACE2-1","SPACE3-1","SPACE4-1","SPACE5-1","PLENUM-1"], #["Environment"],#
    resample="1h",
    title=f"{variable} (Hourly Mean)"
)

In [ ]:
util.plot_sql_series([
    # {"kind":"var","name":"Zone Air CO2 Concentration","key":"SPACE1-1","label":"CO2 SPACE1-1"},
    {"kind":"var","name":"Air System Outdoor Air Mass Flow Rate","key":"*","label":"OA ṁ SPACE1-1"},
    # system-level node is also helpful (replace with your OA node key if different):
    {"kind":"var","name":"System Node Mass Flow Rate","key":"*","label":"OA node V̇"},
], reporting_freq=("TimeStep","Hourly"), resample="15min", meters_to_kwh=False)

In [ ]:
# What “Zone … Outdoor Air …” style vars exist?
util.list_sql_zone_variables(like="Zone %Outdoor Air%")

# Node-based flow variables (system-level). Then skim keys that look like OA nodes.
util.list_sql_zone_variables(name="System Node Mass Flow Rate")
util.list_sql_zone_variables(name="System Node Standard Density Volume Flow Rate")

# Controller/airloop scalar:
util.list_sql_zone_variables(name="Air System Outdoor Air Flow Fraction")


In [ ]:
# 1) Name of the schedule you actuate (created by prepare_run_with_co2)
sched = getattr(util, "_co2_outdoor_schedule", "CO2-Outdoor-Actuated")
print("Schedule:", sched)

# 2) Ensure SQL is produced and ask E+ to record that schedule’s value each timestep
util.ensure_output_sqlite()
util.ensure_output_variables([{
    "name": "Schedule Value",   # this is the reporting variable name
    "key":  sched,              # must match the schedule's name exactly
    "freq": "TimeStep",         # or "Hourly" if you prefer
}], activate=True, reset=True)

# 3) Run a quick design-day (or annual if you prefer)
util.run_design_day()  # or util.run_annual()

# 4) Plot from SQL
fig = util.plot_sql_series(
    selections=[{
        "kind": "var",
        "name": "Schedule Value",
        "key":  sched,
        "label": "Outdoor CO₂ [ppm]",
    }],
    reporting_freq=("TimeStep",),   # match what you requested
    include_design_days=False,
    resample="1H",                  # None for raw
    meters_to_kwh=False,
    title="Outdoor CO₂ Schedule (Actuated)",
    show=True,
)

In [ ]:
util.list_sql_zone_variables(name="Air System Outdoor Air Mass Flow Rate")

In [ ]:
# 3) See what meters actually have rows
for m in output_meteres:
    display(util.inspect_sql_meter(m, include_design_days=True))

# 4) Plot site electricity (facility)
elect_fig=util.plot_sql_meters(
    output_meteres,
    reporting_freq=("TimeStep","Hourly"),
    include_design_days=False,
    resample="1h",               # sum to hourly kWh
    meters_to_kwh=True,
    title=f"{', '.join(output_meteres)}"
)

# (optional) Net purchased if you enabled those two meters:
# elect_purchased=util.plot_sql_net_purchased_electricity(resample="1h")

In [ ]:
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,              # don't filter; show all
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)

In [ ]:
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,              # don't filter; show all
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)
occ_keys

In [ ]:
occ_keys

In [ ]:
fig = util.plot_sql_series(
    selections=output_sels,
    reporting_freq=None,      # pull whatever is in the DB
    resample='1h',            # average to hourly; set to None for native timestep
    aggregate_vars='mean',    # hourly mean occupancy; use 'sum' for person-hours per hour
    title='Occupant Count per Zone',
    show=True
)

###Control

In [ ]:

txt = ex.list_available_api_data_csv(util.state).decode("utf-8", errors="ignore")

In [ ]:
txt

In [ ]:
api_catalog_df(util)['METERS']

In [ ]:
[*api_catalog_df(util)]

In [ ]:
def api_catalog_df(self, *, save_csv: bool = False) -> dict[str, "pd.DataFrame"]:
    """
    Discover **runtime API–exposed catalogs** from EnergyPlus and return them as
    pandas DataFrames, grouped by section.

    Under the hood this wraps:
        self.api.exchange.list_available_api_data_csv(self.state)

    What you get
    ------------
    A dict mapping **section name → DataFrame**, for *all* sections present in
    the current model / E+ build. Typical keys you may see:
      - "ACTUATORS"
      - "INTERNAL_VARIABLES"
      - "PLUGIN_GLOBAL_VARIABLES"
      - "TRENDS"
      - "METERS"
      - "VARIABLES"

    Notes & scope
    -------------
    • This catalog comes **directly from the runtime API** (no IDF parsing, no RDD/MDD/EDD).
    • Availability depends on when you call it; best after inputs are parsed or API data are ready.
      Use one of:
         - inside `callback_after_get_input`, or
         - after warmup via `callback_after_new_environment_warmup_complete`, or
         - when `self.api.exchange.api_data_fully_ready(self.state)` is True.
    • Column shapes vary slightly across sections / versions. This function assigns
      sensible headers per known section and pads/truncates rows as needed.

    Parameters
    ----------
    save_csv : bool, default False
        If True, writes the **raw** CSV from EnergyPlus to `<out_dir>/api_catalog.csv`.

    Returns
    -------
    dict[str, pandas.DataFrame]
        A dictionary of DataFrames keyed by section name. Missing sections simply won't appear.

    Examples
    --------
    >>> # Get everything the runtime reports
    >>> sections = util.api_catalog_df()
    >>> list(sections.keys())
    ['ACTUATORS', 'INTERNAL_VARIABLES', 'PLUGIN_GLOBAL_VARIABLES', 'TRENDS', 'METERS', 'VARIABLES']

    >>> # Inspect schedule-based actuators you can set via get_actuator_handle(...)
    >>> acts = sections.get("ACTUATORS", pd.DataFrame())
    >>> acts.query("ComponentType == 'Schedule:Compact' and ControlType == 'Schedule Value'").head()

    >>> # See available report variables (names/keys/units) the API knows about
    >>> vars_df = sections.get("VARIABLES", pd.DataFrame())
    >>> vars_df.head()

    >>> # Save the raw catalog for auditing
    >>> util.api_catalog_df(save_csv=True)
    """
    import os
    import pandas as pd

    ex = self.api.exchange
    csv_bytes = ex.list_available_api_data_csv(self.state)

    # Optionally persist the raw CSV
    if save_csv:
        try:
            out_path = os.path.join(self.out_dir, "api_catalog.csv")
            with open(out_path, "wb") as f:
                f.write(csv_bytes)
            try:
                self._log(1, f"[api_catalog] Saved → {out_path} ({len(csv_bytes)} bytes)")
            except Exception:
                print(f"[api_catalog] Saved → {out_path} ({len(csv_bytes)} bytes)")
        except Exception:
            pass

    # Parse the catalog: the file is a sequence of sections, each starting with "**NAME**"
    lines = csv_bytes.decode("utf-8", errors="replace").splitlines()
    sections_raw: dict[str, list[list[str]]] = {}
    current = None
    for raw in lines:
        line = raw.strip()
        if not line:
            continue
        if line.startswith("**") and line.endswith("**"):
            current = line.strip("*").strip().upper().replace(" ", "_")
            sections_raw.setdefault(current, [])
            continue
        # Catalog rows are simple CSV without quoted commas → split on ','
        row = [c.strip() for c in line.split(",")]
        if current:
            sections_raw[current].append(row)

    # Known schemas per section (fallbacks are applied when row lengths differ)
    SCHEMAS: dict[str, list[str]] = {
        # Example row: Actuator,Schedule:Compact,Schedule Value,OCCUPY-1,[ ]
        "ACTUATORS": ["Kind", "ComponentType", "ControlType", "ActuatorKey", "Units"],
        # Example row: Internal Variable,Zone,Zone Floor Area,LIVING ZONE,[m2]
        "INTERNAL_VARIABLES": ["Kind", "VariableType", "VariableName", "KeyValue", "Units"],
        # Example row: Plugin Global Variable,<name>
        "PLUGIN_GLOBAL_VARIABLES": ["Kind", "Name"],
        # Example row: Trend,<name>,<length> (varies)
        "TRENDS": ["Kind", "Name", "Length"],
        # Example row: Meter,Electricity:Facility,[J] (varies)
        "METERS": ["Kind", "MeterName", "Units"],
        # Example row: Variable,Zone Mean Air Temperature,LIVING ZONE,[C] (varies)
        "VARIABLES": ["Kind", "VariableName", "KeyValue", "Units"],
    }

    dfs: dict[str, pd.DataFrame] = {}
    for sec, rows in sections_raw.items():
        # Choose schema or a generic fallback wide enough for the observed rows
        cols = SCHEMAS.get(sec)
        if cols is None:
            max_cols = max([len(r) for r in rows] + [5])
            cols = [f"col{i+1}" for i in range(max_cols)]

        # Normalize rows to the column count
        width = len(cols)
        norm = [(r + [""] * (width - len(r)))[:width] for r in rows]
        df = pd.DataFrame(norm, columns=cols)

        # Light cleanup
        if "Kind" in df.columns:
            df["Kind"] = df["Kind"].astype(str).str.strip().str.title()
        for c in df.columns:
            df[c] = df[c].astype(str).str.strip()

        dfs[sec] = df

    return dfs

txt

In [ ]:
ex.list_available_api_data_csv(util.state)

In [ ]:
util.api.api.getAPIData(api.state)

In [ ]:
ex = util.api.exchange

In [ ]:
dir(util.api.api.getAPIData)

## Energy+ Documentation

https://energyplus.net/assets/nrel_custom/pdfs/pdfs_v25.1.0/EngineeringReference.pdf

https://energyplus.net/assets/nrel_custom/pdfs/pdfs_v24.1.0/InputOutputReference.pdf